In [2]:
include("../../L-BFGS/OracleFunction.jl")
include("../../L-BFGS/LBFGS.jl")
include("../../utilities/genFunc.jl")
using .LBFGS
using .OracleFunction
using LinearAlgebra, BenchmarkTools, CSV, DataFrames

baseDir = joinpath("../", "results/LBFGS/3D/")
mkpath(baseDir);

## 3D PLOTS

In [ ]:
### time with respect to lambda and epsilon

# parameters
lambdas = 10. .^(-15:6)
epss    = 10. .^(-15:-6)
memsize = 7
m       = 300
n       = 20


# output csv
outputvsc = joinpath(baseDir, "statisticsLBFGS-lambda-eps-m" * string(m) * "n" * string(n) * "--time.csv");
accData = Dict(
    :lambda   => Array{Float64}(undef, 0),
    :epsilon  => Array{Float64}(undef, 0),
    :memsize  => Array{Int64}(undef, 0),
    :meantime => Array{Float64}(undef, 0),
    :stdtime  => Array{Float64}(undef, 0)
    )

# computation
for (λ, ϵ) ∈ Iterators.product(lambdas, epss) |> collect
    ls = genFunc(:exactRandDataset, λ=λ, m=m, n=n) |> LeastSquaresF

    t = @benchmark begin
        _ = LimitedMemoryBFGS($ls, ϵ=$ϵ, m=memsize)
    end samples=10 evals=1

    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:meantime], mean(t.times))
    push!(accData[:stdtime], std(t.times))
    println("Done: λ " * string(λ) * " - ϵ " * string(ϵ))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

In [ ]:
### iterations with respect to lambda and epsilon

# parameters
lambdas = 10. .^(-15:6)
epss    = 10. .^(-15:-6)
memsize = 7
maxIter = 10000
m       = 300
n       = 20


# output csv
outputvsc = joinpath(baseDir, "statisticsLBFGS-lambda-eps-m" * string(m) * "n" * string(n) * "--iterations.csv");
accData = Dict(
    :lambda     => Array{Float64}(undef, 0),
    :epsilon    => Array{Float64}(undef, 0),
    :memsize    => Array{Int64}(undef, 0),
    :iterations => Array{Int64}(undef, 0)
    )

# computation
for (λ, ϵ) ∈ Iterators.product(lambdas, epss) |> collect
    ls = genFunc(:exactRandDataset, λ=λ, m=m, n=n) |> LeastSquaresF

    t = maxIter - LimitedMemoryBFGS(ls, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)[:RemainingEvaluations]

    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:iterations], t)
    println("Done: λ " * string(λ) * " - ϵ " * string(ϵ))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

In [ ]:
### error with respect to lambda and epsilon

# parameters
lambdas = 10. .^(-15:6)
epss    = 10. .^(-15:-6)
memsize = 7
maxIter = 10000
m       = 300
n       = 20


# output csv
outputvsc = joinpath(baseDir, "statisticsLBFGS-lambda-eps-m" * string(m) * "n" * string(n) * "--error.csv");
accData = Dict(
    :lambda   => Array{Float64}(undef, 0),
    :epsilon  => Array{Float64}(undef, 0),
    :memsize  => Array{Int64}(undef, 0),
    :relative => Array{Float64}(undef, 0),
    :residual => Array{Float64}(undef, 0)
    )

# computation
for (λ, ϵ) ∈ Iterators.product(lambdas, epss) |> collect
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)

    t = LimitedMemoryBFGS(ls, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)

    relative_error = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
    residual = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])

    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:relative], relative_error)
    push!(accData[:residual], residual)
    println("Done: λ " * string(λ) * " - ϵ " * string(ϵ))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

In [ ]:
### iterations with respect to epsilon and memory size

# parameters
λ        = 10^-3
epss     = 10. .^(-15:-6)
memsizes = [1, 3, 5, 7, 9, 11, 15, 20, 40, 60]
maxIter  = 10000
m        = 300
n        = 20


# output csv
outputvsc = joinpath(baseDir, "statisticsLBFGS-eps-mem-m" * string(m) * "n" * string(n) * "--iterations.csv");
accData = Dict(
    :lambda     => Array{Float64}(undef, 0),
    :epsilon    => Array{Float64}(undef, 0),
    :memsize    => Array{Int64}(undef, 0),
    :iterations => Array{Int64}(undef, 0)
    )

# computation
for (ϵ, memsize) ∈ Iterators.product(epss, memsizes) |> collect
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)

    t = maxIter - LimitedMemoryBFGS(ls, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)[:RemainingEvaluations]

    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:iterations], t)
    println("Done: mem " * string(memsize) * " - ϵ " * string(ϵ))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

In [ ]:
### error with respect to epsilon and memory size

# parameters
λ        = 10^-3
epss     = 10. .^(-15:-6)
memsizes = [1, 3, 5, 7, 9, 11, 15, 20, 40, 60]
maxIter  = 10000
m        = 300
n        = 20


# output csv
outputvsc = joinpath(baseDir, "statisticsLBFGS-eps-mem-m" * string(m) * "n" * string(n) * "--error.csv");
accData = Dict(
    :lambda   => Array{Float64}(undef, 0),
    :epsilon  => Array{Float64}(undef, 0),
    :memsize  => Array{Int64}(undef, 0),
    :relative => Array{Float64}(undef, 0),
    :residual => Array{Float64}(undef, 0)
    )

# computation
for (ϵ, memsize) ∈ Iterators.product(epss, memsizes) |> collect
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)

    t = LimitedMemoryBFGS(ls, ϵ=ϵ, MaxEvaluations=maxIter, m=memsize)

    relative_error = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
    residual = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])

    push!(accData[:lambda], λ)
    push!(accData[:epsilon], ϵ)
    push!(accData[:memsize], memsize)
    push!(accData[:relative], relative_error)
    push!(accData[:residual], residual)
    println("Done: mem " * string(memsize) * " - ϵ " * string(ϵ))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));